<a href="https://colab.research.google.com/github/chanheum/machinerunning/blob/main/with_hugging_face_240708.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!python --version

In [ ]:
!pip install mxnet==1.9.1
!pip install gluonnlp==0.10.0
!pip install pandas==1.5.3
!pip install tqdm==4.66.1
!pip install sentencepiece==0.1.99
!pip install transformers==4.34.0
!pip install torch==2.0.1+cu118
!pip install numpy==1.23.1

In [ ]:
!pip install firebase-admin
!pip show firebase-admin

In [ ]:
!pip install firebase_functions

python version : 3.10.12

<Modules Version>
mxnet : 1.9.1

gluonnlp : 0.10.0

pandas : 1.5.3

tqdm : 4.66.1

sentencepiece : 0.1.99

transformers : 4.34.0

torch : 2.0.1+cu118

In [ ]:
# ★
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
import time

In [ ]:
# ★ Hugging Face를 통한 모델 및 토크나이저 Import
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [ ]:
# GPU 사용 시
device = torch.device("cuda:0")

In [ ]:
# ★
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# 데이터 불러오기
# !wget https://www.dropbox.com/s/374ftkec978br3d/ratings_train.txt?dl=1
# !wget https://www.dropbox.com/s/977gbwh542gdy94/ratings_test.txt?dl=1

# 데이터 불러오기 (나)
import pandas as pd
# chatbot_data = pd.read_csv('/content/drive/MyDrive/sentences.csv', encoding='cp949')
chatbot_data = pd.read_csv('/content/drive/MyDrive/sentences_custom_240708.csv', encoding='cp949')      # 파일명 수정

In [ ]:
chatbot_data.loc[(chatbot_data['의도'] == "neutral"), '의도'] = 0          #일상대화  => 0
chatbot_data.loc[(chatbot_data['의도'] == "call"), '의도'] = 1          #코웨이 AI 호출  => 1
chatbot_data.loc[(chatbot_data['의도'] == "power_on"), '의도'] = 2      #전원 on => 2
chatbot_data.loc[(chatbot_data['의도'] == "power_off"), '의도'] = 3     #전원 off  => 3
chatbot_data.loc[(chatbot_data['의도'] == "mode_goodmoring"), '의도'] = 4       #굿모닝모드 => 4
chatbot_data.loc[(chatbot_data['의도'] == "mode_goodday"), '의도'] = 5      #굿데이모드  => 5
chatbot_data.loc[(chatbot_data['의도'] == "mode_goodnight"), '의도'] = 6       #굿나잇모드 => 6
chatbot_data.loc[(chatbot_data['의도'] == "recline_zerog"), '의도'] = 7      #무중력모드 => 7
chatbot_data.loc[(chatbot_data['의도'] == "recline_swing"), '의도'] = 8       #스윙모드 => 8
chatbot_data.loc[(chatbot_data['의도'] == "heat_on"), '의도'] = 9       #온열 on => 9
chatbot_data.loc[(chatbot_data['의도'] == "heat_off"), '의도'] = 10      #온열 off  => 10

In [ ]:
chatbot_data.sample(n=10)

In [ ]:
data_list = []
for q, label in zip(chatbot_data['발화문'], chatbot_data['의도'])  :
    if(type(q) != float):
        data = []
        data.append(q)
        data.append(str(label))

        data_list.append(data)
    else:
        pass

In [ ]:
print(len(data_list))
print(data_list[0])
# print(data_list[6000])
# print(data_list[12000])
print(data_list[-1])

In [ ]:
#train & test 데이터로 나누기
from sklearn.model_selection import train_test_split

dataset_train, dataset_test = train_test_split(data_list, test_size=0.25, shuffle=True, random_state=34)

In [ ]:
print(len(dataset_train))
print((dataset_train))

In [ ]:
print(len(dataset_test))
print(dataset_test)

for data, label in dataset_test:
    print(type(data))
    print(data)
    print(label)

In [ ]:
# ★
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer,vocab, max_len,
                 pad, pair):

        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len,vocab=vocab, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))


    def __len__(self):
        return (len(self.labels))

In [ ]:
# Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [ ]:
# ★
tok = tokenizer.tokenize

data_train = BERTDataset(dataset_train, 0, 1, tok, vocab, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, vocab, max_len, True, False)
print(data_train[0])
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes = 11, # 클래스 수 조정
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate

        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)

    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)

        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [ ]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [ ]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [ ]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
train_dataloader

In [ ]:
# 머신러닝 학습시작
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

In [ ]:
# 최초 1회 설정
import firebase_admin
from firebase_admin import credentials
# The Cloud Functions for Firebase SDK to create Cloud Functions and set up triggers.
from firebase_functions import db_fn, https_fn

# The Firebase Admin SDK to access the Firebase Realtime Database.
from firebase_admin import initialize_app, db

cred = credentials.Certificate('/content/drive/MyDrive/coway-chair-test-server-firebase-adminsdk-hhu3y-2c4d064bcf.json')

try:
    app = firebase_admin.initialize_app(cred, { 'databaseURL' : "https://coway-chair-test-server-default-rtdb.firebaseio.com/" })
except ValueError:
    app = firebase_admin.initialize_app    # 그대로 admin 사용

In [ ]:
# firebase에 데이터 쓰기
# dir = db.reference()
# dir.update({"Result" : "testtest"})

In [ ]:
# firebase에 데이터 읽어오기
dir = db.reference('Result')
print(dir.get())

In [ ]:
from typing import Any
@db_fn.on_value_created(reference="/Result")
def makeuppercase(event: db_fn.Event[Any]) -> None:
    """Listens for new messages added to /messages/{pushId}/original and
    creates an uppercase version of the message to /messages/{pushId}/uppercase
    """

    # Grab the value that was written to the Realtime Database.
    original = event.data
    if not isinstance(original, str):
        print(f"Not a string: {event.reference}")
        return

    # Use the Admin SDK to set an "uppercase" sibling.
    print(f"Uppercasing {event.params['pushId']}: {original}")
    upper = original.upper()
    parent = db.reference(event.reference).parent
    if parent is None:
        print("Message can't be root node.")
        return
    parent.child("uppercase").set(upper)

In [ ]:
from firebase_functions.core import CloudEvent
@db_fn.on_value_written(reference="hello/world")
def onwriteexample(event: db_fn.Event[db_fn.Change]) -> None:
    print("Hello from db write event:", event)


@db_fn.on_value_created(reference="hello/world")
def oncreatedexample(event: db_fn.Event) -> None:
    print("Hello from db create event:", event)


@db_fn.on_value_deleted(reference="hello/world")
def ondeletedexample(event: db_fn.Event) -> None:
    print("Hello from db delete event:", event)


@db_fn.on_value_updated(reference="hello/world")
def onupdatedexample(event: db_fn.Event[db_fn.Change]) -> None:
    print("Hello from db updated event:", event)

In [ ]:
#토큰화
# tokenizer = tokenizer.tokenize
# tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

def predict(predict_sentence):

    if predict_sentence == '0':
        return

    # firebase에 데이터 쓰기
    dir = db.reference()
    data = [predict_sentence, '0']
    dataset_another = [data]
    print(dataset_another)
    another_test = BERTDataset(dataset_another, 0, 1, tok, vocab, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)

    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)

        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 1:
                test_eval.append("사용자 코웨이 AI 호출")

                dir.child("NLU").update({"Command" : "null"})
                dir.child("NLU").update({"Result" : "COWAY"})

            elif np.argmax(logits) == 2:
                test_eval.append("안마의자 전원 ON 명령")

                dir.child("NLU").update({"Command" : "null"})
                dir.child("NLU").update({"Result" : "POWER_ON"})

            elif np.argmax(logits) == 3:
                test_eval.append("안마의자 전원 OFF 명령")

                dir.child("NLU").update({"Command" : "null"})
                dir.child("NLU").update({"Result" : "POWER_OFF"})

            elif np.argmax(logits) == 4:
                test_eval.append("Good Moring모드 동작 명령")

                dir.child("NLU").update({"Command" : "null"})
                dir.child("NLU").update({"Result" : "GOOD_MORING"})

            elif np.argmax(logits) == 5:
                test_eval.append("Good Day모드 동작 명령")

                dir.child("NLU").update({"Command" : "null"})
                dir.child("NLU").update({"Result" : "GOOD_DAY"})

            elif np.argmax(logits) == 6:
                test_eval.append("Good night모드 동작 명령")

                dir.child("NLU").update({"Command" : "null"})
                dir.child("NLU").update({"Result" : "GOOD_NIGHT"})

            elif np.argmax(logits) == 7:
                test_eval.append("무중력모드 동작 명령")

                dir.child("NLU").update({"Command" : "null"})
                dir.child("NLU").update({"Result" : "ZEROG"})

            elif np.argmax(logits) == 8:
                test_eval.append("스윙모드 동작 명령")

                dir.child("NLU").update({"Command" : "null"})
                dir.child("NLU").update({"Result" : "SWING"})

            elif np.argmax(logits) == 9:
                test_eval.append("온열 기능 ON 명령")

                dir.child("NLU").update({"Command" : "null"})
                dir.child("NLU").update({"Result" : "HEAT_ON"})

            elif np.argmax(logits) == 10:
                test_eval.append("온열 기능 OFF 명령")

                dir.child("NLU").update({"Command" : "null"})
                dir.child("NLU").update({"Result" : "HEAT_OFF"})

            else:
                test_eval.append("일상 대화")
        print(test_eval[0])

In [ ]:
#질문 무한반복하기! 0 입력시 종료
end = 1
data = ""
data_old = ""
flag = 1
while end == 1 :
    time.sleep(1)
    # firebase에 데이터 읽어오기
    dir = db.reference().child("NLU").child("Command")
    data = dir.get()

    if(data_old != data):
        data_old = data

        if data == '0' and flag == 0:
            print("종료합니다.")
            end = 0
        elif data == "null":
            pass
        else:
            try:
                flag = 0
                predict(data)
            except TypeError:
                pass
    # sentence = input("하고싶은 말을 입력해주세요 : ")
    # if sentence == '0' :
    #     print("종료합니다.")
    #     end = 0
    # predict(sentence)
    # print("\n")